In [15]:
import pandas as pd
import numpy as np
import os
target_directory = "Z:\\FNS_GA_v5_100_gen_CNN_hyperparam_work\\models\\total_flux\\data"
pre_process_data = ["tf_times_source_training_data.csv", "tf_times_source_validation_data.csv"]
void_value = 1.
variable_cassette_2a = True

In [16]:
os.chdir(target_directory)

In [17]:
#def encode(data):
#    print('Shape of data (BEFORE encode): %s' % str(data.shape))
#    encoded = to_categorical(data)
#    print('Shape of data (AFTER  encode): %s\n' % str(encoded.shape))
#    return encoded

In [18]:
#def one_hot_encode_fns(material_value, number_of_materials):
#    zeros_ = np.zeros(number_of_materials)
#    material_array = zeros_
#    material_array[int(material_value)-1] = 1
#    print(material_array)
#one_hot_encode_fns(material_value=4, number_of_materials = 4)

In [19]:
def preprocess_variable_cassette_raw_ga_data(data_file, output_file_name="", fixed_plate_count=120):
    with open(data_file, 'r') as data_file_obj:
        data_master_list = []
        for line in data_file_obj:
            list_ = line.split(",")

            # print(len(list_))
            # print(list_)
            new_list = []
            for value in list_:
                if value.strip() == "":
                    continue
                new_list.append(value.strip())
            # print(len(new_list), len(new_list) - fixed_plate_count - 1)
            # print(new_list)
            size_of_all_patterns = len(new_list) - 1

            size_of_variable_cassette = size_of_all_patterns - fixed_plate_count

            new_list_with_variable_cassette = []
            for count, value in enumerate(new_list):
                new_list_with_variable_cassette.append(value)
                if count == 120:
                    for _ in range(30 - size_of_variable_cassette):
                        new_list_with_variable_cassette.append('1')
            # print("final list", len(new_list_with_variable_cassette), new_list_with_variable_cassette)
            data_master_list.append(new_list_with_variable_cassette)
        # print(len(data_master_list))
        with open(output_file_name, 'w') as output_file_obj:
            for _ in data_master_list:
                write_string = ""
                for __ in _:
                    write_string +=  str(__) + ","
                output_file_obj.write(write_string[:-1] + "\n")

In [20]:
for preproc_ in pre_process_data:
    preprocess_variable_cassette_raw_ga_data(data_file = preproc_, output_file_name = "processed_3x3_corrected_" + preproc_)

In [21]:
def return_pattern(pat_a, pat_b, pat_c, plate_number):
    row_1 = np.array([pat_c[plate_number], pat_b[plate_number],
                     pat_b[plate_number], pat_a[plate_number]])
    return np.reshape(row_1, (2,2))


In [22]:
#data_ = np.loadtxt("rep_validation_gen_46.csv", delimiter=",")
#X = data_[:,0:150]
#Y = data_[:,150:151]
#zones = [1, 2, 3]
#number_of_plates = 60

def build_3x3_fns_data(data_file_string, zones = [1, 2, 3], number_of_plates = 60, number_of_materials = 4):
    data_ = np.loadtxt(data_file_string, delimiter=",")
    X = data_[:,1:151]
    Y = data_[:,0:1]
    print(X[0])
    print(Y[0])
    pattern_shapes_list = []
    for fns_pattern_count, pattern in enumerate(X): 

        ### Setting patterns for this dataset
        if variable_cassette_2a == False:
            pattern_1a = pattern[0:10]
            pattern_1b = pattern[10:30]
            pattern_1c = pattern[30:50]
            pattern_2a = pattern[50:70]
            pattern_2b = pattern[70:90]
            pattern_2c = pattern[90:110]
            pattern_3a = np.array([void_value, void_value, void_value, void_value, void_value,
                                   void_value, void_value, void_value, void_value, void_value,
                                   void_value, void_value, void_value, void_value, void_value,
                                   void_value, void_value, void_value, void_value, void_value])
            pattern_3b = pattern[110:130]
            pattern_3c = pattern[130:150]

            np_four_10_array = np.array([void_value, void_value, void_value, void_value, void_value,
                                         void_value, void_value, void_value, void_value, void_value])
            pattern_1a = np.concatenate((np_four_10_array, pattern_1a))
        
        else:
            pattern_1b = pattern[0:20]
            pattern_1c = pattern[20:40]
            pattern_2b = pattern[40:60]
            pattern_2c = pattern[60:80]

            pattern_3b = pattern[80:100]
            pattern_3c = pattern[100:120]

            np_four_10_array = np.array([void_value, void_value, void_value, void_value, void_value,
                                         void_value, void_value, void_value, void_value, void_value])
            pattern_1a = pattern[120:130]
            pattern_1a = np.concatenate((np_four_10_array, pattern_1a))
            pattern_2a = pattern[130:150]
            pattern_3a = np.array([void_value, void_value, void_value, void_value, void_value,
                                   void_value, void_value, void_value, void_value, void_value,
                                   void_value, void_value, void_value, void_value, void_value,
                                   void_value, void_value, void_value, void_value, void_value])

        for plate in range(number_of_plates):
            if plate == 0:
                pattern_array = return_pattern(pattern_1a, pattern_1b, pattern_1c, plate)
                continue

            if plate < 20:
                pattern_ = return_pattern(pattern_1a, pattern_1b, pattern_1c, plate)
                pattern_array = np.concatenate((pattern_array, pattern_))
            if plate < 40 and plate >= 20:
                pattern_ = return_pattern(pattern_2a, pattern_2b, pattern_2c, plate-20)
                pattern_array = np.concatenate((pattern_array, pattern_))
            if plate >= 40:
                pattern_ = return_pattern(pattern_3a, pattern_3b, pattern_3c, plate-40)
                pattern_array = np.concatenate((pattern_array, pattern_))

        pattern_array_reshaped =  np.reshape(pattern_array, (60, 2, 2))
        #print(pattern_array_reshaped.shape)
        ### One hot encoding the 3x3x60 array into a 3x3x60x5 array. Stackoverflow magic:
        ### https://stackoverflow.com/questions/36960320/convert-a-2d-matrix-to-a-3d-one-hot-matrix-numpy
        ### 4 types of materials is hard coded inside np.arange(#)
        pattern_array_reshaped_1he = (np.arange(number_of_materials) == pattern_array_reshaped[...,None]-1).astype(int)

        pattern_shapes_list.append(np.array(pattern_array_reshaped_1he))
        #pattern_array_reshaped_1he_array = np.append(pattern_array_reshaped_1he_array, pattern_array_reshaped_1he)
        print(fns_pattern_count, pattern_array_reshaped_1he.shape)
    pattern_shapes_array = np.array(pattern_shapes_list)
    return pattern_shapes_array, Y

In [23]:
for data in pre_process_data:
    
    pattern_shapes_array, Y = build_3x3_fns_data("processed_"+data)
    np.save("X_"+data, pattern_shapes_array)
    np.save("Y_"+data, Y)

[3. 4. 2. 3. 3. 2. 2. 4. 4. 2. 2. 2. 4. 2. 3. 4. 2. 2. 2. 4. 2. 3. 4. 3.
 2. 2. 4. 2. 3. 2. 4. 4. 4. 4. 4. 4. 4. 3. 2. 4. 3. 2. 3. 4. 4. 4. 3. 4.
 2. 2. 4. 4. 3. 2. 4. 4. 4. 4. 2. 2. 4. 3. 3. 3. 4. 4. 4. 4. 3. 4. 3. 4.
 4. 3. 4. 3. 2. 3. 4. 2. 3. 4. 3. 4. 3. 2. 4. 4. 2. 4. 4. 4. 4. 3. 2. 4.
 2. 3. 4. 2. 4. 3. 3. 3. 4. 3. 2. 2. 4. 2. 3. 4. 4. 4. 2. 4. 2. 4. 4. 2.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 2.]
[14929520.]
0 (60, 2, 2, 4)
1 (60, 2, 2, 4)
2 (60, 2, 2, 4)
3 (60, 2, 2, 4)
4 (60, 2, 2, 4)
5 (60, 2, 2, 4)
6 (60, 2, 2, 4)
7 (60, 2, 2, 4)
8 (60, 2, 2, 4)
9 (60, 2, 2, 4)
10 (60, 2, 2, 4)
11 (60, 2, 2, 4)
12 (60, 2, 2, 4)
13 (60, 2, 2, 4)
14 (60, 2, 2, 4)
15 (60, 2, 2, 4)
16 (60, 2, 2, 4)
17 (60, 2, 2, 4)
18 (60, 2, 2, 4)
19 (60, 2, 2, 4)
20 (60, 2, 2, 4)
21 (60, 2, 2, 4)
22 (60, 2, 2, 4)
23 (60, 2, 2, 4)
24 (60, 2, 2, 4)
25 (60, 2, 2, 4)
26 (60, 2, 2, 4)
27 (60, 2, 2, 4)
28 (60, 2, 2, 4)
29 (60, 2, 2, 4)
30 (60, 2, 2, 4)
31 (60, 2, 2,

487 (60, 2, 2, 4)
488 (60, 2, 2, 4)
489 (60, 2, 2, 4)
490 (60, 2, 2, 4)
491 (60, 2, 2, 4)
492 (60, 2, 2, 4)
493 (60, 2, 2, 4)
494 (60, 2, 2, 4)
495 (60, 2, 2, 4)
496 (60, 2, 2, 4)
497 (60, 2, 2, 4)
498 (60, 2, 2, 4)
499 (60, 2, 2, 4)
500 (60, 2, 2, 4)
501 (60, 2, 2, 4)
502 (60, 2, 2, 4)
503 (60, 2, 2, 4)
504 (60, 2, 2, 4)
505 (60, 2, 2, 4)
506 (60, 2, 2, 4)
507 (60, 2, 2, 4)
508 (60, 2, 2, 4)
509 (60, 2, 2, 4)
510 (60, 2, 2, 4)
511 (60, 2, 2, 4)
512 (60, 2, 2, 4)
513 (60, 2, 2, 4)
514 (60, 2, 2, 4)
515 (60, 2, 2, 4)
516 (60, 2, 2, 4)
517 (60, 2, 2, 4)
518 (60, 2, 2, 4)
519 (60, 2, 2, 4)
520 (60, 2, 2, 4)
521 (60, 2, 2, 4)
522 (60, 2, 2, 4)
523 (60, 2, 2, 4)
524 (60, 2, 2, 4)
525 (60, 2, 2, 4)
526 (60, 2, 2, 4)
527 (60, 2, 2, 4)
528 (60, 2, 2, 4)
529 (60, 2, 2, 4)
530 (60, 2, 2, 4)
531 (60, 2, 2, 4)
532 (60, 2, 2, 4)
533 (60, 2, 2, 4)
534 (60, 2, 2, 4)
535 (60, 2, 2, 4)
536 (60, 2, 2, 4)
537 (60, 2, 2, 4)
538 (60, 2, 2, 4)
539 (60, 2, 2, 4)
540 (60, 2, 2, 4)
541 (60, 2, 2, 4)
542 (60, 2

943 (60, 2, 2, 4)
944 (60, 2, 2, 4)
945 (60, 2, 2, 4)
946 (60, 2, 2, 4)
947 (60, 2, 2, 4)
948 (60, 2, 2, 4)
949 (60, 2, 2, 4)
950 (60, 2, 2, 4)
951 (60, 2, 2, 4)
952 (60, 2, 2, 4)
953 (60, 2, 2, 4)
954 (60, 2, 2, 4)
955 (60, 2, 2, 4)
956 (60, 2, 2, 4)
957 (60, 2, 2, 4)
958 (60, 2, 2, 4)
959 (60, 2, 2, 4)
960 (60, 2, 2, 4)
961 (60, 2, 2, 4)
962 (60, 2, 2, 4)
963 (60, 2, 2, 4)
964 (60, 2, 2, 4)
965 (60, 2, 2, 4)
966 (60, 2, 2, 4)
967 (60, 2, 2, 4)
968 (60, 2, 2, 4)
969 (60, 2, 2, 4)
970 (60, 2, 2, 4)
971 (60, 2, 2, 4)
972 (60, 2, 2, 4)
973 (60, 2, 2, 4)
974 (60, 2, 2, 4)
975 (60, 2, 2, 4)
976 (60, 2, 2, 4)
977 (60, 2, 2, 4)
978 (60, 2, 2, 4)
979 (60, 2, 2, 4)
980 (60, 2, 2, 4)
981 (60, 2, 2, 4)
982 (60, 2, 2, 4)
983 (60, 2, 2, 4)
984 (60, 2, 2, 4)
985 (60, 2, 2, 4)
986 (60, 2, 2, 4)
987 (60, 2, 2, 4)
988 (60, 2, 2, 4)
989 (60, 2, 2, 4)
990 (60, 2, 2, 4)
991 (60, 2, 2, 4)
992 (60, 2, 2, 4)
993 (60, 2, 2, 4)
994 (60, 2, 2, 4)
995 (60, 2, 2, 4)
996 (60, 2, 2, 4)
997 (60, 2, 2, 4)
998 (60, 2

1402 (60, 2, 2, 4)
1403 (60, 2, 2, 4)
1404 (60, 2, 2, 4)
1405 (60, 2, 2, 4)
1406 (60, 2, 2, 4)
1407 (60, 2, 2, 4)
1408 (60, 2, 2, 4)
1409 (60, 2, 2, 4)
1410 (60, 2, 2, 4)
1411 (60, 2, 2, 4)
1412 (60, 2, 2, 4)
1413 (60, 2, 2, 4)
1414 (60, 2, 2, 4)
1415 (60, 2, 2, 4)
1416 (60, 2, 2, 4)
1417 (60, 2, 2, 4)
1418 (60, 2, 2, 4)
1419 (60, 2, 2, 4)
1420 (60, 2, 2, 4)
1421 (60, 2, 2, 4)
1422 (60, 2, 2, 4)
1423 (60, 2, 2, 4)
1424 (60, 2, 2, 4)
1425 (60, 2, 2, 4)
1426 (60, 2, 2, 4)
1427 (60, 2, 2, 4)
1428 (60, 2, 2, 4)
1429 (60, 2, 2, 4)
1430 (60, 2, 2, 4)
1431 (60, 2, 2, 4)
1432 (60, 2, 2, 4)
1433 (60, 2, 2, 4)
1434 (60, 2, 2, 4)
1435 (60, 2, 2, 4)
1436 (60, 2, 2, 4)
1437 (60, 2, 2, 4)
1438 (60, 2, 2, 4)
1439 (60, 2, 2, 4)
1440 (60, 2, 2, 4)
1441 (60, 2, 2, 4)
1442 (60, 2, 2, 4)
1443 (60, 2, 2, 4)
1444 (60, 2, 2, 4)
1445 (60, 2, 2, 4)
1446 (60, 2, 2, 4)
1447 (60, 2, 2, 4)
1448 (60, 2, 2, 4)
1449 (60, 2, 2, 4)
1450 (60, 2, 2, 4)
1451 (60, 2, 2, 4)
1452 (60, 2, 2, 4)
1453 (60, 2, 2, 4)
1454 (60, 2,

1834 (60, 2, 2, 4)
1835 (60, 2, 2, 4)
1836 (60, 2, 2, 4)
1837 (60, 2, 2, 4)
1838 (60, 2, 2, 4)
1839 (60, 2, 2, 4)
1840 (60, 2, 2, 4)
1841 (60, 2, 2, 4)
1842 (60, 2, 2, 4)
1843 (60, 2, 2, 4)
1844 (60, 2, 2, 4)
1845 (60, 2, 2, 4)
1846 (60, 2, 2, 4)
1847 (60, 2, 2, 4)
1848 (60, 2, 2, 4)
1849 (60, 2, 2, 4)
1850 (60, 2, 2, 4)
1851 (60, 2, 2, 4)
1852 (60, 2, 2, 4)
1853 (60, 2, 2, 4)
1854 (60, 2, 2, 4)
1855 (60, 2, 2, 4)
1856 (60, 2, 2, 4)
1857 (60, 2, 2, 4)
1858 (60, 2, 2, 4)
1859 (60, 2, 2, 4)
1860 (60, 2, 2, 4)
1861 (60, 2, 2, 4)
1862 (60, 2, 2, 4)
1863 (60, 2, 2, 4)
1864 (60, 2, 2, 4)
1865 (60, 2, 2, 4)
1866 (60, 2, 2, 4)
1867 (60, 2, 2, 4)
1868 (60, 2, 2, 4)
1869 (60, 2, 2, 4)
1870 (60, 2, 2, 4)
1871 (60, 2, 2, 4)
1872 (60, 2, 2, 4)
1873 (60, 2, 2, 4)
1874 (60, 2, 2, 4)
1875 (60, 2, 2, 4)
1876 (60, 2, 2, 4)
1877 (60, 2, 2, 4)
1878 (60, 2, 2, 4)
1879 (60, 2, 2, 4)
1880 (60, 2, 2, 4)
1881 (60, 2, 2, 4)
1882 (60, 2, 2, 4)
1883 (60, 2, 2, 4)
1884 (60, 2, 2, 4)
1885 (60, 2, 2, 4)
1886 (60, 2,

2280 (60, 2, 2, 4)
2281 (60, 2, 2, 4)
2282 (60, 2, 2, 4)
2283 (60, 2, 2, 4)
2284 (60, 2, 2, 4)
2285 (60, 2, 2, 4)
2286 (60, 2, 2, 4)
2287 (60, 2, 2, 4)
2288 (60, 2, 2, 4)
2289 (60, 2, 2, 4)
2290 (60, 2, 2, 4)
2291 (60, 2, 2, 4)
2292 (60, 2, 2, 4)
2293 (60, 2, 2, 4)
2294 (60, 2, 2, 4)
2295 (60, 2, 2, 4)
2296 (60, 2, 2, 4)
2297 (60, 2, 2, 4)
2298 (60, 2, 2, 4)
2299 (60, 2, 2, 4)
2300 (60, 2, 2, 4)
2301 (60, 2, 2, 4)
2302 (60, 2, 2, 4)
2303 (60, 2, 2, 4)
2304 (60, 2, 2, 4)
2305 (60, 2, 2, 4)
2306 (60, 2, 2, 4)
2307 (60, 2, 2, 4)
2308 (60, 2, 2, 4)
2309 (60, 2, 2, 4)
2310 (60, 2, 2, 4)
2311 (60, 2, 2, 4)
2312 (60, 2, 2, 4)
2313 (60, 2, 2, 4)
2314 (60, 2, 2, 4)
2315 (60, 2, 2, 4)
2316 (60, 2, 2, 4)
2317 (60, 2, 2, 4)
2318 (60, 2, 2, 4)
2319 (60, 2, 2, 4)
2320 (60, 2, 2, 4)
2321 (60, 2, 2, 4)
2322 (60, 2, 2, 4)
2323 (60, 2, 2, 4)
2324 (60, 2, 2, 4)
2325 (60, 2, 2, 4)
2326 (60, 2, 2, 4)
2327 (60, 2, 2, 4)
2328 (60, 2, 2, 4)
2329 (60, 2, 2, 4)
2330 (60, 2, 2, 4)
2331 (60, 2, 2, 4)
2332 (60, 2,

2761 (60, 2, 2, 4)
2762 (60, 2, 2, 4)
2763 (60, 2, 2, 4)
2764 (60, 2, 2, 4)
2765 (60, 2, 2, 4)
2766 (60, 2, 2, 4)
2767 (60, 2, 2, 4)
2768 (60, 2, 2, 4)
2769 (60, 2, 2, 4)
2770 (60, 2, 2, 4)
2771 (60, 2, 2, 4)
2772 (60, 2, 2, 4)
2773 (60, 2, 2, 4)
2774 (60, 2, 2, 4)
2775 (60, 2, 2, 4)
2776 (60, 2, 2, 4)
2777 (60, 2, 2, 4)
2778 (60, 2, 2, 4)
2779 (60, 2, 2, 4)
2780 (60, 2, 2, 4)
2781 (60, 2, 2, 4)
2782 (60, 2, 2, 4)
2783 (60, 2, 2, 4)
2784 (60, 2, 2, 4)
2785 (60, 2, 2, 4)
2786 (60, 2, 2, 4)
2787 (60, 2, 2, 4)
2788 (60, 2, 2, 4)
2789 (60, 2, 2, 4)
2790 (60, 2, 2, 4)
2791 (60, 2, 2, 4)
2792 (60, 2, 2, 4)
2793 (60, 2, 2, 4)
2794 (60, 2, 2, 4)
2795 (60, 2, 2, 4)
2796 (60, 2, 2, 4)
2797 (60, 2, 2, 4)
2798 (60, 2, 2, 4)
2799 (60, 2, 2, 4)
2800 (60, 2, 2, 4)
2801 (60, 2, 2, 4)
2802 (60, 2, 2, 4)
2803 (60, 2, 2, 4)
2804 (60, 2, 2, 4)
2805 (60, 2, 2, 4)
2806 (60, 2, 2, 4)
2807 (60, 2, 2, 4)
2808 (60, 2, 2, 4)
2809 (60, 2, 2, 4)
2810 (60, 2, 2, 4)
2811 (60, 2, 2, 4)
2812 (60, 2, 2, 4)
2813 (60, 2,